In [8]:
from modules.preprocessing import get_dataset, balanced_datasets
from modules.distances import distance, radius, radius_distance
from modules.single_classifier import SingleClassifier
from modules.ensamble_classifier import EnsambleClassifier
from modules.ulits import softmax

from tqdm import tqdm
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
df = get_dataset('data/original_texts.csv')
df_90texts, df_400texts = balanced_datasets(df)
print(len(df_90texts), len(df_400texts))

train_90texts, test_90texts = train_test_split(df_90texts, train_size=0.8,  random_state=42)
train_400texts, test_400texts = train_test_split(df_400texts, train_size=0.8, random_state=42)

100%|████████████████████████████████████████████| 8/8 [02:17<00:00, 17.22s/it]


810 1600


In [3]:
train_90texts.head()

,native,language_background,text,len,word_tokens,words_truncated,word_unigrams,word_bigrams,word_trigrams,len_sym,char_truncated,character 3-grams,character 4-grams,character 5-grams,character 6-grams,character 7-grams,character 8-grams,character 9-grams,character 10-grams,num_texts
id,,,,,,,,,,,,,,,,,,,,
6571,swe,FL,Мой герои 15 лет я работал журналистом и в т...,678,"[мой, герои, 15, лет, я, работал, журналистом,...","[мой, герои, 15, лет, я, работал, журналистом,...","{'я': 4, 'с': 3, 'встречи': 3, 'и': 2, 'в': 2,...","{'были с': 2, 'мой герои': 1, 'герои 15': 1, '...","{'мой герои 15': 1, 'герои 15 лет': 1, '15 лет...",4695,Мой герои 15 лет я работал журналистом и в т...,"{'али': 7, 'ли ': 7, 'стр': 6, 'ист': 5, ' вс'...","{' вст': 5, 'встр': 5, 'стре': 5, ' жур': 4, '...","{' встр': 5, 'встре': 5, ' журн': 4, 'журна': ...","{' встре': 5, ' журна': 4, 'журнал': 4, 'урнал...","{' журнал': 4, 'журнали': 4, 'урналис': 4, 'рн...","{' журнали': 4, 'журналис': 4, 'урналист': 4, ...","{' журналис': 4, 'журналист': 4, 'урналисто': ...","{' журналист': 4, 'журналисто': 2, 'урналистом...",105
3702,fr,FL,"Лоль В. Штейн родилась здесь, в городе Тале, и...",118,"[лоль, в, штейн, родилась, здесь, в, городе, т...","[лоль, в, штейн, родилась, здесь, в, городе, т...","{'в': 7, 'ее': 3, 'лоль': 2, 'штейн': 2, 'я': ...","{'лоль в': 2, 'в штейн': 2, 'штейн родилась': ...","{'лоль в штейн': 2, 'в штейн родилась': 1, 'шт...",783,"Лоль В. Штейн родилась здесь, в городе Тале, и...","{' в ': 6, ' не': 6, 'ли ': 6, 'оль': 5, 'не '...","{' не ': 5, ' они': 4, 'они ': 4, ' ее ': 3, '...","{' они ': 4, ', они': 3, 'Лоль ': 2, 'оль В': ...","{', они ': 3, 'Лоль В': 2, 'оль В.': 2, 'ль В....","{'Лоль В.': 2, 'оль В. ': 2, 'ль В. Ш': 2, 'ь ...","{'Лоль В. ': 2, 'оль В. Ш': 2, 'ль В. Шт': 2, ...","{'Лоль В. Ш': 2, 'оль В. Шт': 2, 'ль В. Ште': ...","{'Лоль В. Шт': 2, 'оль В. Ште': 2, 'ль В. Штей...",269
4166,fin,HL,Каждый человек ищет себе счастье.А что счастье...,154,"[каждый, человек, ищет, себе, счастье, а, что,...","[каждый, человек, ищет, себе, счастье, а, что,...","{'счастье': 4, 'и': 3, 'это': 3, 'каждый': 2, ...","{'счастье это': 2, 'каждый человек': 1, 'челов...","{'каждый человек ищет': 1, 'человек ищет себе'...",1071,Каждый человек ищет себе счастье.А что счастье...,"{'сть': 8, 'ть ': 8, 'аст': 7, 'час': 6, 'тье'...","{'част': 6, 'асть': 6, 'стье': 6, 'тье ': 5, '...","{'часть': 6, 'астье': 6, 'стье ': 5, 'аждый': ...","{'частье': 6, 'астье ': 5, 'аждый ': 4, ' чело...","{'частье ': 5, ' челове': 4, 'человек': 4, ' с...","{' человек': 4, ' счастье': 3, '.Счастье': 3, ...","{'.Счастье ': 3, 'е счастье': 2, ' счастье ': ...","{' каждый де': 2, 'каждый ден': 2, 'аждый день...",1129
3836,kaz,HL,Моя бабушка...Будто этих словах описана вся мо...,180,"[моя, бабушка, будто, этих, словах, описана, в...","[моя, бабушка, будто, этих, словах, описана, в...","{'моя': 5, 'бабушка': 4, 'мне': 3, 'на': 3, 'ж...","{'моя бабушка': 3, 'во мне': 2, 'бабушка будто...","{'моя бабушка будто': 1, 'бабушка будто этих':...",1159,Моя бабушка...Будто этих словах описана вся мо...,"{' мо': 8, 'оя ': 7, ' ба': 7, 'баб': 7, 'абу'...","{' баб': 7, 'бабу': 7, 'абуш': 7, 'бушк': 7, '...","{' бабу': 7, 'бабуш': 7, 'абушк': 7, 'бушка': ...","{' бабуш': 7, 'бабушк': 7, 'абушка': 6, 'оя ба...","{' бабушк': 7, 'бабушка': 6, 'оя бабу': 5, 'я ...","{' бабушка': 6, 'оя бабуш': 5, 'я бабушк': 5, ...","{'оя бабушк': 5, 'я бабушка': 5, ' бабушка ': ...","{'оя бабушка': 5, ' моя бабуш': 4, 'моя бабушк...",454
3769,kaz,HL,"Бабушка, без сомнения, один из главных людей в...",496,"[бабушка, без, сомнения, один, из, главных, лю...","[бабушка, без, сомнения, один, из, главных, лю...","{'меня': 4, 'они': 3, 'и': 3, 'в': 2, 'жизни':...","{'бабушка без': 1, 'без сомнения': 1, 'сомнени...","{'бабушка без сомнения': 1, 'без сомнения один...",3227,"Бабушка, без сомнения, один из главных людей в...","{' по': 6, ' вс': 6, 'все': 6, ' и ': 5, 'ми '...","{' все': 6, 'абуш': 4, 'бушк': 4, ' мен': 4, '...","

In [5]:
def clean_tqdm():
    for instance in list(tqdm._instances): 
        tqdm._decr_instances(instance)

In [ ]:
# TODO Пофиксить баланс классов и переделать :(

# Language Background - df_90texts

## Word_unigrams

In [6]:
classes = list(train_90texts['language_background'].unique())  # ['HL', 'FL']
y_true = list(train_90texts['language_background'])[:100] # len = 810
p_lengths = [200, 500, 1000, 1500, 2000, 2500, 3000]
profs = list(train_90texts['word_unigrams'])[:100]  # list of Counter

In [7]:
clean_tqdm()
ec = EnsambleClassifier([profs], y_true, p_lengths, classes=classes)
classifiers = ec.classifiers
y_pred = ec.forward_ensamble(confidence=False)

100%|███████████████████████████████████████████| 7/7 [12:51<00:00, 110.18s/it]


In [9]:
Counter(y_pred)

Counter({'FL': 100})

In [10]:
accuracy_score (y_true, y_pred)

0.55

## Word_bigrams

In [12]:
clean_tqdm()
profs = list(train_90texts['word_bigrams'])[:100]
ec = EnsambleClassifier([profs], y_true, p_lengths, classes=classes)
classifiers = ec.classifiers
y_pred = ec.forward_ensamble(confidence=False)

100%|███████████████████████████████████████████| 7/7 [18:43<00:00, 160.49s/it]


In [13]:
Counter(y_pred)

Counter({'HL': 61, 'FL': 39})

In [14]:
accuracy_score (y_true, y_pred)

0.58

## Word_trigrams

In [15]:
clean_tqdm()
profs = list(train_90texts['word_trigrams'])[:100]
ec = EnsambleClassifier([profs], y_true, p_lengths, classes=classes)
classifiers = ec.classifiers
y_pred = ec.forward_ensamble(confidence=False)

100%|███████████████████████████████████████████| 7/7 [17:34<00:00, 150.63s/it]


In [16]:
Counter(y_pred)

Counter({'HL': 31, 'FL': 69})

In [17]:
accuracy_score (y_true, y_pred)

0.58

## Character 3_grams

In [18]:
clean_tqdm()
profs = list(train_90texts['character 3-grams'])[:100]
ec = EnsambleClassifier([profs], y_true, p_lengths, classes=classes)
classifiers = ec.classifiers
y_pred = ec.forward_ensamble(confidence=False)

print(Counter(y_pred))

accuracy_score (y_true, y_pred)

100%|████████████████████████████████████████| 7/7 [2:15:24<00:00, 1160.59s/it]


Counter({'FL': 99, 'HL': 1})


0.56

## Character 4_grams

In [20]:
clean_tqdm()
profs = list(train_90texts['character 4-grams'])[:100]
ec = EnsambleClassifier([profs], y_true, p_lengths, classes=classes)
classifiers = ec.classifiers
y_pred = ec.forward_ensamble(confidence=False)

print(Counter(y_pred))

accuracy_score (y_true, y_pred)

Accuracy: 0.66 (from Google Colab)

## Character 5_grams

In [ ]:
clean_tqdm()
profs = list(train_90texts['character 5-grams'])[:100]
ec = EnsambleClassifier([profs], y_true, p_lengths, classes=classes)
classifiers = ec.classifiers
y_pred = ec.forward_ensamble(confidence=False)

print(Counter(y_pred))

accuracy_score (y_true, y_pred)

Accuracy: 0.66 (from Google Colab)

## Character 6_grams

In [ ]:
clean_tqdm()
profs = list(train_90texts['character 6-grams'])[:100]
ec = EnsambleClassifier([profs], y_true, p_lengths, classes=classes)
classifiers = ec.classifiers
y_pred = ec.forward_ensamble(confidence=False)

print(Counter(y_pred))

accuracy_score (y_true, y_pred)

Accuracy: 0.69 (from Google Colab)

## Character 7_grams

In [ ]:
clean_tqdm()
profs = list(train_90texts['character 7-grams'])[:100]
ec = EnsambleClassifier([profs], y_true, p_lengths, classes=classes)
classifiers = ec.classifiers
y_pred = ec.forward_ensamble(confidence=False)

print(Counter(y_pred))

accuracy_score (y_true, y_pred)

Accuracy: 0.68 (from Google Colab)

## Character 8_grams

In [ ]:
clean_tqdm()
profs = list(train_90texts['character 4-grams'])[:100]
ec = EnsambleClassifier([profs], y_true, p_lengths, classes=classes)
classifiers = ec.classifiers
y_pred = ec.forward_ensamble(confidence=False)

print(Counter(y_pred))

accuracy_score (y_true, y_pred)

Accuracy: 0.62 (from Google Colab)

## Character 9_grams

In [ ]:
clean_tqdm()
profs = list(train_90texts['character 4-grams'])[:100]
ec = EnsambleClassifier([profs], y_true, p_lengths, classes=classes)
classifiers = ec.classifiers
y_pred = ec.forward_ensamble(confidence=False)

print(Counter(y_pred))

accuracy_score (y_true, y_pred)

Accuracy: 0.63 (from Google Colab)

## Character 10_grams

In [ ]:
clean_tqdm()
profs = list(train_90texts['character 4-grams'])[:100]
ec = EnsambleClassifier([profs], y_true, p_lengths, classes=classes)
classifiers = ec.classifiers
y_pred = ec.forward_ensamble(confidence=False)

print(Counter(y_pred))

accuracy_score (y_true, y_pred)

Accuracy: 0.64 (from Google Colab)

# Language Background - df_400texts

In [21]:
classes = list(train_400texts['language_background'].unique())  # ['HL', 'FL']
y_true = list(train_400texts['language_background'])[:100] # len = 810
p_lengths = [200, 500, 1000, 1500, 2000, 2500, 3000]

## Word_unigrams

In [22]:
clean_tqdm()

profs = list(train_400texts['word_unigrams'])[:100]  # list of Counter
ec = EnsambleClassifier([profs], y_true, p_lengths, classes=classes)
classifiers = ec.classifiers
y_pred = ec.forward_ensamble(confidence=False)

print(Counter(y_pred))

accuracy_score (y_true, y_pred)

100%|███████████████████████████████████████████| 7/7 [16:08<00:00, 138.30s/it]


Counter({'FL': 91, 'HL': 9})


0.54

## Word_bigrams

In [23]:
clean_tqdm()

profs = list(train_400texts['word_bigrams'])[:100]  # list of Counter
ec = EnsambleClassifier([profs], y_true, p_lengths, classes=classes)
classifiers = ec.classifiers
y_pred = ec.forward_ensamble(confidence=False)

print(Counter(y_pred))

accuracy_score (y_true, y_pred)

100%|███████████████████████████████████████████| 7/7 [14:11<00:00, 121.70s/it]


Counter({'FL': 66, 'HL': 34})


0.65

## Word_trigrams 

In [24]:
clean_tqdm()

profs = list(train_400texts['word_trigrams'])[:100]  # list of Counter
ec = EnsambleClassifier([profs], y_true, p_lengths, classes=classes)
classifiers = ec.classifiers
y_pred = ec.forward_ensamble(confidence=False)

print(Counter(y_pred))
 
accuracy_score (y_true, y_pred)

100%|███████████████████████████████████████████| 7/7 [14:19<00:00, 122.75s/it]


Counter({'FL': 54, 'HL': 46})


0.69

## Character 3-grams

In [25]:
clean_tqdm()

profs = list(train_400texts['character 3-grams'])[:100]  # list of Counter
ec = EnsambleClassifier([profs], y_true, p_lengths, classes=classes)
classifiers = ec.classifiers
y_pred = ec.forward_ensamble(confidence=False)

print(Counter(y_pred))

accuracy_score (y_true, y_pred)

100%|█████████████████████████████████████████| 7/7 [1:21:35<00:00, 699.35s/it]


Counter({'HL': 59, 'FL': 41})


0.78

## Character 4-grams

In [26]:
clean_tqdm()

profs = list(train_400texts['character 4-grams'])[:100]  # list of Counter
ec = EnsambleClassifier([profs], y_true, p_lengths, classes=classes)
classifiers = ec.classifiers
y_pred = ec.forward_ensamble(confidence=False)

print(Counter(y_pred))

accuracy_score (y_true, y_pred)

100%|█████████████████████████████████████████| 7/7 [1:53:14<00:00, 970.62s/it]


Counter({'HL': 79, 'FL': 21})


0.72

## Character 5-grams

In [ ]:
clean_tqdm()

profs = list(train_400texts['character 5-grams'])[:100]  # list of Counter
ec = EnsambleClassifier([profs], y_true, p_lengths, classes=classes)
classifiers = ec.classifiers
y_pred = ec.forward_ensamble(confidence=False)

print(Counter(y_pred))

accuracy_score (y_true, y_pred)

Accuracy: 0.41 (from Google Colab)

## Character 6-grams

In [ ]:
clean_tqdm()

profs = list(train_400texts['character 6-grams'])[:100]  # list of Counter
ec = EnsambleClassifier([profs], y_true, p_lengths, classes=classes)
classifiers = ec.classifiers
y_pred = ec.forward_ensamble(confidence=False)

print(Counter(y_pred))

accuracy_score (y_true, y_pred)

Accuracy: 0.41 (from Google Colab)

## Character 7-grams

In [ ]:
clean_tqdm()

profs = list(train_400texts['character 7-grams'])[:100]  # list of Counter
ec = EnsambleClassifier([profs], y_true, p_lengths, classes=classes)
classifiers = ec.classifiers
y_pred = ec.forward_ensamble(confidence=False)

print(Counter(y_pred))

accuracy_score (y_true, y_pred)

Accuracy: 0.4 (from Google Colab)

## Character 8-grams

In [ ]:
clean_tqdm()

profs = list(train_400texts['character 8-grams'])[:100]  # list of Counter
ec = EnsambleClassifier([profs], y_true, p_lengths, classes=classes)
classifiers = ec.classifiers
y_pred = ec.forward_ensamble(confidence=False)

print(Counter(y_pred))

accuracy_score (y_true, y_pred)

Accuracy: 0.38 (from Google Colab)

## Character 9-grams

In [ ]:
clean_tqdm()

profs = list(train_400texts['character 9-grams'])[:100]  # list of Counter
ec = EnsambleClassifier([profs], y_true, p_lengths, classes=classes)
classifiers = ec.classifiers
y_pred = ec.forward_ensamble(confidence=False)

print(Counter(y_pred))

accuracy_score (y_true, y_pred)

Accuracy: 0.42 (from Google Colab)

## Character 10-grams

In [ ]:
clean_tqdm()

profs = list(train_400texts['character 10-grams'])[:100]  # list of Counter
ec = EnsambleClassifier([profs], y_true, p_lengths, classes=classes)
classifiers = ec.classifiers
y_pred = ec.forward_ensamble(confidence=False)

print(Counter(y_pred))

accuracy_score (y_true, y_pred)

Accuracy: 0.45 (from Google Colab)

# Running best models on test data

In [34]:
y_tr = list(train_90texts['language_background'])[:100]
y_te = list(test_90texts['language_background'])[:100]

Counter(y_tr), Counter(y_te)

(Counter({'FL': 50, 'HL': 50}), Counter({'HL': 33, 'FL': 67}))

In [30]:
# 90texts - character 6-grams
clean_tqdm()
profs = list(test_90texts['character 6-grams'])[:100]
ec = EnsambleClassifier([profs], y_true, p_lengths, classes=classes)
classifiers = ec.classifiers
y_pred = ec.forward_ensamble(confidence=False)
y_true = list(test_90texts['language_background'])[:100] # len = 810

print(Counter(y_pred))

accuracy_score (y_true, y_pred)

Counter({'FL': 73, 'HL': 27})


0.49

In [ ]:
# 400texts - character 3-grams
clean_tqdm()

profs = list(train_400texts['character 3-grams'])[:100]  # list of Counter
ec = EnsambleClassifier([profs], y_true, p_lengths, classes=classes)
classifiers = ec.classifiers
y_pred = ec.forward_ensamble(confidence=False)
y_true = list(test_400texts['language_background'])[:100] # len = 810
print(Counter(y_pred))

accuracy_score (y_true, y_pred)